이론 정리 페이지 : https://www.notion.so/leew-work/CH05-3-2cee0bd14001433cb5979d99e8ef6d21

In [1]:
from tensorflow import keras

### LSTM

keras에서 제공하는 IMDB를 사용할 예정  
- 표준 어휘를 사용해 영화 리뷰를 정수 배열로 변환한 데이터셋  
- 감정 분류하는 방법 학습 -> 새로운 텍스트에 대해 동일한 예 실행시켜 모델 검증 

문장들 : 감정이 긍정 <-> 부정을 나타내는 레이블과 함께 정수 배열 형태로 제공됨 

In [2]:
# keras 라이브러리, 그리고 imdb 데이터셋 로드
from keras.preprocessing import sequence
from keras.models import Sequential 
from keras.layers import Dense, Embedding 
from keras.layers import LSTM 
from keras.datasets import imdb

In [4]:
# 데이터셋에서 로드할 단어의 최대 수 지정 
max_features = 20000

# 문장의 최대 단어 수와 batch 크기 지정 
maxlen = 50
batch_size = 32

# 데이터 로드 
print('  Loading data...   ')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

# 문장의 패딩 
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

  Loading data...   
x_train shape: (25000, 50)
x_test shape: (25000, 50)


이제 데이터를 탐색하려고 한다. 